# 다중클래스 분류 모델 
### 데이터 셋 준비 
- 로이터에서 제공하는 뉴스와이어 데이터셋을 이용
- 이 데이터셋은 총 11,228개의 샘플로 구성
- 라벨은 46개 주제로 지정되어 0에서 45의 값을 가지고 있음
- 데이터셋은 이미 정수로 인코딩되어 있으며, 정수값은 단어의 빈도수를 나타냄 
- 모든 단어를 고려할 수 없으므로 빈도수가 높은 단어를 위주로 데이터셋을 생성
- 15,000번째로 많이 사용하는 단어까지만 데이터셋으로 만들고 싶다면, num_words 인자에 15000이라고 지정하면 됨

In [21]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dropout

In [4]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=15000)

### 데이터 셋 구성 
- 훈련셋 8,982개와 시험셋 2,246개로 구성된 총 11,228개 샘플이 로딩
- 훈련셋과 시험셋의 비율은 load_data() 함수의 test_split 인자로 조절 가능 함
- 각 샘플은 뉴스 한 건을 의미하며, 단어의 인덱스로 구성되어 있습니다. 
- num_words=20000으로 지정했기 때문에 빈도수가 15,000을 넘는 단어는 로딩되지 않음
- 훈련셋 8,982개 중 다시 7,000개을 훈련셋으로 나머지를 검증셋으로 분리

In [6]:
x_train = x_train[:7000]
y_train = y_train[:7000]
x_val = x_train[7000:]
y_val = y_train[7000:]

### 샘플 길이 맞추기 
- 각 샘플의 길이가 달라서 모델의 입력으로 사용하기 위해 케라스에서 제공되는 전처리 함수인 sequence의 pad_sequences() 함수를 사용
- 이 함수는 두 가지 역할을 수행함
    - 문장의 길이를 maxlen 인자로 맞춰줌 
    - 예를 들어 120으로 지정했다면 120보다 짧은 문장은 0으로 채워서 120단어로 맞춰주고 120보다 긴 문장은 120단어까지만 잘라냄
    - (num_samples, num_timesteps)으로 2차원의 numpy 배열로 만들어줍니다. 
    - maxlen을 120으로 지정하였다면, num_timesteps도 120이 됩니다.

In [8]:
x_train = sequence.pad_sequences(x_train, maxlen=120)
x_val = sequence.pad_sequences(x_val, maxlen=120)
x_test = sequence.pad_sequences(x_test, maxlen=120)

### 다층퍼셉트론 신경망 모델
- 임베딩 레이어는 0에서 45의 정수값으로 지정된 단어를 128벡터로 인코딩합니다. 
- 문장의 길이가 120이므로 임베딩 레이어는 128 속성을 가진 벡터를 120개 반환합니다. 
- 이를 Flatten 레이어를 통해 1차원 벡터로 만든 뒤 전결합층으로 전달합니다. 
- 46개 주제를 분류해야 하므로 출력층의 활성화 함수로 ‘softmax’를 사용하였습니다.

In [11]:
model = Sequential()
model.add(Embedding(15000, 128, input_length=120))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(46, activation='softmax'))

### 순환 신경망 모델
- 임베딩 레이어에서 반환되는 120개 벡터를 LSTM의 타입스텝으로 입력하는 모델
- LSTM의 input_dim은 임베딩 레이어에서 인코딩된 벡터크기인 128입니다.

In [14]:
model = Sequential()
model.add(Embedding(15000, 128))
model.add(LSTM(128))
model.add(Dense(46, activation='softmax'))

### 컨볼루션 신경망 모델
- 임베딩 레이어에서 반환되는 120개 벡터를 컨볼루션 필터를 적용한 모델
- 필터크기가 3인 컨볼루션 레이어는 120개의 벡터를 입력받아 118개의 벡터를 반환
- 벡터 크기는 컨볼루션 레이어를 통과하면서 128개에서 256개로 늘어남
- 글로벌 맥스풀링 레이어는 입력되는 118개 벡터 중 가장 큰 벡터 하나를 반환
- 그 벡터 하나를 전결합층을 통해서 다중클래스를 분류

In [20]:
model = Sequential()
model.add(Embedding(15000, 128, input_length=120))
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(46, activation='softmax'))

### 순환 컨볼루션 신경망 모델
- 컨볼루션 레이어에서 나온 특징벡터들을 맥스풀링(MaxPooling1D)를 통해 1/4로 줄여준 다음 LSTM의 입력으로 넣어주는 모델
- 컨볼루션 레이어에서 반환한 118개의 벡터를 1/4로 줄여서 29개를 반환
- 따라서 LSTM 레이어의 timesteps는 49개가 됩니다
- 참고로 input_dim은 256

In [24]:
model = Sequential()
model.add(Embedding(15000, 128, input_length=120))
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(128))
model.add(Dense(46, activation='softmax'))